In [27]:
#imports

%matplotlib inline
import pandas as pd
import numpy as np
import time
import requests
import bs4
import json
from flatten_json import flatten

In [28]:
# 20 requests every 1 seconds(s)
# 100 requests every 2 minutes(s)

apikey = "RGAPI-2a11ccd9-3521-4259-8999-2bc2c8dab556"

In [29]:
# get challenger summonerId from league-exp-api
# 1.2 delay bc short list

req = requests.get("https://na1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GRANDMASTER/I?page=1&api_key={}"
                   .format(apikey),time.sleep(1.2))
assert req.status_code == 200
data = req.json()
sumids = [d['summonerId'] for d in data]

In [30]:
# get accountId via summonerId via summoner-api
# 1.2 delay bc short list

accids = []
for sumid in sumids:
    sumreq = requests.get("https://na1.api.riotgames.com/lol/summoner/v4/summoners/{}?api_key={}"
                          .format(sumid,apikey),time.sleep(1.2))
    assert sumreq.status_code == 200
    accids.append(sumreq.json()['accountId'])

In [31]:
# get game ids from match history via match-api
# 1.3 delay bc long list

gamids = []

for acc in accids:
    matlreq = requests.get("https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/{}?api_key={}"
                           .format(acc,apikey),time.sleep(1.3))
    assert matlreq.status_code == 200
    gamids += [d['gameId'] for d in matlreq.json()['matches']]

In [32]:
# print original length, then correct overlap from shared games and print new length

print(len(gamids))

gamids = set(gamids)

print(len(gamids))

20500
14557


In [35]:
with open('chall_gam_id_list.txt', 'r') as f:
    challgames = f.read().splitlines()

In [46]:
challgames =set([int(e) for e in challgames])

In [47]:
len(challgames)

10446

In [48]:
len(gamids)

14557

In [52]:
len(gamids - (challgames & gamids))

10724

In [53]:
gamids = gamids - (challgames & gamids)

In [54]:
# get individual match information from gamids list via match-api
# 1.5 delay bc long list and repeated issues with 

matchlist = []

for gam in gamids:
    gamreq = requests.get("https://na1.api.riotgames.com/lol/match/v4/matches/{}?api_key={}"
                          .format(gam,apikey),time.sleep(1.3))
    
    i=0
    while gamreq.status_code != 200:
        gamreq = requests.get("https://na1.api.riotgames.com/lol/match/v4/matches/{}?api_key={}"
                          .format(gam,apikey),time.sleep(1.3))
        i+=1
        if i == 5:
            break
            
    match = flatten(gamreq.json())
    matchlist.append(match)

In [55]:
gamreq.status_code

200

In [56]:
# check length of matchlist 

check = [len(m) for m in matchlist]
len(matchlist),max(check),min(check)

(10724, 1550, 133)

In [57]:
# create df from matches

df = pd.DataFrame(matchlist)

In [68]:
gmgames = list(df['gameId'])

In [69]:
with open('gm_gam_id_list.txt', 'w') as f:
    for item in gmgames:
        f.write("%s\n" % item)

In [61]:
len(set(list(df['gameId'])))

10724

In [63]:
compression_opts = dict(method='zip',
                        archive_name='gm.csv')  

In [64]:
df.to_csv('gm.zip', index=False,
          compression=compression_opts)